In [25]:
%%time
%load_ext autoreload
%autoreload 2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM, GRU
import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CPU times: user 579 µs, sys: 394 µs, total: 973 µs
Wall time: 645 µs


In [27]:
from RainPredictionMachine.data import CleanDataRpm
cleaner = CleanDataRpm()
df = cleaner.clean_data(1)
df

,Chuva,Pres,Pres_max,Pres_min,Radiacao,Temp,Temp_orvalho,Temp_max,Temp_min,Temp_orvalho_max,...,Umid_min,Umid,Dir_vento,Rajada_vento,Vel_vento,Latitude,Longitude,Altitude,datahora,classe_chuva
0,0.0,923.5,923.5,922.4,0.0,21.8,18.9,22.8,21.7,19.3,...,79.0,84.0,164.0,2.6,0.7,-23.523889,-46.869444,776.5,2020-01-01 00:00:00+00:00,0
1,0.0,924.0,924.0,923.5,0.0,22.0,19.4,22.3,21.6,19.7,...,84.0,85.0,348.0,2.0,0.1,-23.523889,-46.869444,776.5,2020-01-01 01:00:00+00:00,0
2,0.0,924.0,924.1,924.0,0.0,22.1,18.6,22.9,22.0,19.9,...,79.0,81.0,20.0,2.2,0.3,-23.523889,-46.869444,776.5,2020-01-01 02:00:00+00:00,0
3,0.0,923.5,924.1,923.5,0.0,21.4,18.3,22.3,21.4,19.1,...,80.0,82.0,325.0,1.7,0.0,-23.523889,-46.869444,776.5,2020-01-01 03:00:00+00:00,0
4,0.0,922.9,923.5,922.9,0.0,21.4,18.9,21.6,21.1,18.9,...,82.0,86.0,245.0,1.7,1.0,-23.523889,-46.869444,776.5,2020-01-01 04:00:00+00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,0.0,924.9,925.1,924.8,0.0,22.8,18.9,23.4,22.8,19.0,...,72.0,78.0,280.0,0.0,0.0,-23.523889,-46.869444,776.5,2020-12-31 19:00:00+00:00,0
8780,0.0,925.8,925.8,924.9,0.0,21.4,18.6,22.8,21.4,18.9,...,77.0,84.0,211.0,0.0,0.0,-23.523889,-46.869444,776.5,2020-12-31 20:00:00+00:00,0
8781,0.0,926.7,926.7,925.8,0.0,21.3,19.4,21.4,21.3,19.4,...,84.0,89.0,207.0,0.0,0.0,-23.523889,-46.869444,776.5,2020-12-31 21:00:00+00:00,0
8782,0.0,926.9,927.1,926.7,214.4,20.1,18.8,21.3,20.1,19.7,...,89.0,92.0,211.0,0.0,0.0,-23.523889,-46.869444,776.5,2020-12-31 22:00:00+00:00,0


In [3]:
def subsample_sequence(df, length):
    '''returns a random sample sequence from inside a dataframe'''
    index = np.random.randint(0, df.shape[0] - length)
    df_sample = df.iloc[index:index+length]
    
    return df_sample

In [4]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample['Chuva'].iloc[length - 24:]
    
    X_sample = df_subsample.drop(columns= ['classe_chuva', 'datahora'])[0:length -24]
    X_sample = X_sample.values
   
    return np.array(X_sample), np.array(y_sample)

In [5]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    return X, y

In [6]:
train_size = int(df.shape[0]*0.8)
df_train = df.iloc[:train_size,:]
df_test = df.iloc[train_size:,:]

In [7]:
X_train,y_train = get_X_y(df_train,200,72)

In [8]:
X_test,y_test = get_X_y(df_test,200,72)

In [9]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization

norm = Normalization()
norm.adapt(X_train)

2022-03-24 15:18:27.064469: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Arquitetura da rede

In [10]:
model1 = Sequential()

model1.add(norm)

model1.add(LSTM(units=100, activation='tanh',return_sequences=False))
model1.add(Dense(80, activation="tanh"))
model1.add(Dense(50, activation="tanh"))
model1.add(Dense(24, activation="relu"))

model1.compile(loss='mape',
                  optimizer='rmsprop',
                  metrics=['mae','mse','mape'])

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, None, 20)         41        
 n)                                                              
                                                                 
 lstm (LSTM)                 (None, 100)               48400     
                                                                 
 dense (Dense)               (None, 80)                8080      
                                                                 
 dense_1 (Dense)             (None, 50)                4050      
                                                                 
 dense_2 (Dense)             (None, 24)                1224      
                                                                 
Total params: 61,795
Trainable params: 61,754
Non-trainable params: 41
___________________________________________________

In [11]:
model2 = Sequential()

model2.add(norm)

model2.add(LSTM(units=100, activation='tanh',return_sequences=True))
model2.add(LSTM(units=50, activation='tanh',return_sequences=False))
model2.add(Dense(30, activation="tanh"))
model2.add(Dense(24, activation="relu"))

model2.compile(loss='mape',
                  optimizer='rmsprop',
                  metrics=['mae','mse','mape'])

model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, None, 20)         41        
 n)                                                              
                                                                 
 lstm_1 (LSTM)               (None, None, 100)         48400     
                                                                 
 lstm_2 (LSTM)               (None, 50)                30200     
                                                                 
 dense_3 (Dense)             (None, 30)                1530      
                                                                 
 dense_4 (Dense)             (None, 24)                744       
                                                                 
Total params: 80,915
Trainable params: 80,874
Non-trainable params: 41
_________________________________________________

## Ajuste

In [12]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

In [13]:
%%time
model1.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1,
         validation_split=0.2,
          callbacks=[es])

Epoch 1/10
5/5 [==============================] - 2s 105ms/step - loss: 21794372.0000 - mae: 0.1487 - mse: 1.0620 - mape: 21794372.0000 - val_loss: 1415382.5000 - val_mae: 0.1230 - val_mse: 1.1106 - val_mape: 1415382.5000
Epoch 2/10
5/5 [==============================] - 0s 25ms/step - loss: 102042.6641 - mae: 0.1287 - mse: 1.0646 - mape: 102042.6641 - val_loss: 800492.3750 - val_mae: 0.1224 - val_mse: 1.1105 - val_mape: 800492.3750
Epoch 3/10
5/5 [==============================] - 0s 24ms/step - loss: 8552.9805 - mae: 0.1286 - mse: 1.0646 - mape: 8552.9805 - val_loss: 865959.0000 - val_mae: 0.1224 - val_mse: 1.1105 - val_mape: 865959.0000
Epoch 4/10
5/5 [==============================] - 0s 24ms/step - loss: 6.8227 - mae: 0.1286 - mse: 1.0646 - mape: 6.8227 - val_loss: 865959.1875 - val_mae: 0.1224 - val_mse: 1.1105 - val_mape: 865959.1875
Epoch 5/10
5/5 [==============================] - 0s 25ms/step - loss: 6.8227 - mae: 0.1286 - mse: 1.0646 - mape: 6.8227 - val_loss: 865959.0000 - 

In [14]:
%%time
model2.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1,
         validation_split=0.2,
          callbacks=[es])

Epoch 1/10
5/5 [==============================] - 2s 160ms/step - loss: 33973068.0000 - mae: 0.1604 - mse: 1.0628 - mape: 33973068.0000 - val_loss: 3773502.0000 - val_mae: 0.1251 - val_mse: 1.1101 - val_mape: 3773502.0000
Epoch 2/10
5/5 [==============================] - 0s 34ms/step - loss: 1743091.6250 - mae: 0.1302 - mse: 1.0639 - mape: 1743091.6250 - val_loss: 788929.0000 - val_mae: 0.1224 - val_mse: 1.1105 - val_mape: 788929.0000
Epoch 3/10
5/5 [==============================] - 0s 35ms/step - loss: 113781.3906 - mae: 0.1287 - mse: 1.0646 - mape: 113781.3906 - val_loss: 210923.6562 - val_mae: 0.1219 - val_mse: 1.1105 - val_mape: 210923.6562
Epoch 4/10
5/5 [==============================] - 0s 35ms/step - loss: 10668.0674 - mae: 0.1286 - mse: 1.0646 - mape: 10668.0674 - val_loss: 254742.7969 - val_mae: 0.1219 - val_mse: 1.1105 - val_mape: 254742.7969
Epoch 5/10
5/5 [==============================] - 0s 35ms/step - loss: 2982.9487 - mae: 0.1286 - mse: 1.0646 - mape: 2982.9487 - val_

## Avaliacao

In [15]:
y_pred1 = model1.predict(X_test)
y_pred2 = model2.predict(X_test)

In [16]:
model1.evaluate(X_test,y_test)
model2.evaluate(X_test,y_test)

7/7 [==============================] - 0s 10ms/step - loss: 39291.0039 - mae: 0.0942 - mse: 0.3072 - mape: 39291.0039


[39291.00390625, 0.09421201050281525, 0.30723854899406433, 39291.00390625]

In [17]:
df_y_test = pd.DataFrame(y_test).T #cada coluna sera o indice para os dias do futuro -XX dias

In [18]:
df_y_pred1 = pd.DataFrame(y_pred1).T
df_y_pred2 = pd.DataFrame(y_pred2).T

In [19]:
def classe_chuva(precipitacao):
    mm=precipitacao
    if np.isnan(mm):
        chuva = "NaN"
    if mm == 0:
        chuva = 0 #'nao chove'
    elif mm >0 and mm <=5.0:
        chuva = 1 #'fraca'
    elif mm >5.0 and mm<=25.0:
        chuva = 2 #'moderada'
    else:
        chuva = 3 #'forte'
    return chuva

In [20]:
df_y_test_categorical = df_y_test.applymap(classe_chuva)

In [21]:
df_y_pred_categorical1 = df_y_pred1.applymap(classe_chuva)
df_y_pred_categorical2 = df_y_pred2.applymap(classe_chuva)

In [22]:
(df_y_test_categorical == df_y_pred_categorical1).sum() # o quanto acertamos por dia

0      24
1      23
2      10
3      11
4      12
       ..
195    11
196    21
197    11
198    11
199    24
Length: 200, dtype: int64

In [23]:
m1acc = (df_y_test_categorical == df_y_pred_categorical1).sum().sum()/(6000*24)
m2acc = (df_y_test_categorical == df_y_pred_categorical2).sum().sum()/(6000*24)

## Resultado final

In [24]:
print(f'model 1 accuracy: {m1acc}')
print(f'model 2 accuracy: {m2acc}')

model 1 accuracy: 0.02254861111111111
model 2 accuracy: 0.022784722222222224
